In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.linear_model import LogisticRegression as LR

import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_data(path):
    data=np.loadtxt(path)
    X=data[:, 1:]
    y=data[:, 0]
    return X, y

X_train, y_train=get_data('./data/zip.train')
X_test, y_test=get_data('./data/zip.test')
print(X_train.shape, X_test.shape)
print(np.unique(y_train, return_counts=True)[1])
print(np.unique(y_test, return_counts=True)[1])

(7291, 256) (2007, 256)
[1194 1005  731  658  652  556  664  645  542  644]
[359 264 198 166 200 160 170 147 166 177]


In [3]:
fit1 = LDA().fit(X_train, y_train)
fit2 = QDA().fit(X_train, y_train)
fit3 = LR(random_state=0).fit(X_train, y_train)
result1 = fit1.score(X_train, y_train), fit1.score(X_test, y_test)
result2 = fit2.score(X_train, y_train), fit2.score(X_test, y_test)
result3 = fit3.score(X_train, y_train), fit3.score(X_test, y_test)

print(result1)
print(result2)
print(result3)

(0.9380057605266767, 0.885401096163428)
(0.920175558908243, 0.8156452416542103)
(0.9938280071320806, 0.9108121574489287)


In [5]:
!cd /Users/ianfla/Documents/packages
!git clone https://github.com/jlsuarezdiaz/pyDML.git

正克隆到 'pyDML'...
remote: Enumerating objects: 6238, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6238 (delta 0), reused 1 (delta 0), pack-reused 6234
接收对象中: 100% (6238/6238), 215.13 MiB | 13.00 MiB/s, 完成.
处理 delta 中: 100% (2611/2611), 完成.
正在更新文件: 100% (3076/3076), 完成.


In [ ]:
!cd pyDML
!pip3 install -r requirements.txt
!python3 setup.py install